## A: Getting uncertainty from metatrain models

In [ ]:
import ase.build
from metatomic.torch.ase_calculator import MetatomicCalculator
from metatomic.torch import ModelOutput

Before anything, you must download and export the PET-MAD checkpoint that will be used in this tutorial (you can also copy from another directory, if you've downloaded and exported the model already).

```bash
wget https://huggingface.co/lab-cosmo/pet-mad/resolve/v1.0.2/models/pet-mad-v1.0.2.ckpt
mtt export pet-mad-v1.0.2.ckpt -o pet-mad.pt
```

We will now walk you through a demonstration of how you can obtain the uncertainties along with the predictions from a `metatrain` model (which already has the LLPR wrapper trained and prepared). We will use the Atomic Simulation Environment (ASE) for our demonstrations.

Let's start by loading a bulk copper configuration, and the PET-MAD model as a ASE calculator:

In [ ]:
# Build a fcc copper structure
copper = ase.build.bulk("Cu", "fcc", cubic=True)

# Initialize an ASE calculator with a pre-trained potential
calculator = MetatomicCalculator("pet-mad.pt")

Getting the uncertainties from the model is as easy as requesting the uncertainty as a model output, as shown in the following snippet:

In [ ]:
# Get global uncertainty
outputs = calculator.run_model(
    copper,
    outputs={"energy_uncertainty": ModelOutput(unit="eV", per_atom=False)},
)
global_uncertainty = outputs["energy_uncertainty"].block().values.item()
print("Global energy uncertainty (eV):", global_uncertainty)

You can also obtain the "local uncertainties" (square root of the inverse of the local prediction rigidity, LPR, with the same calibration as the global PR) by setting `per_atom=True`:

In [ ]:
# Get local uncertainties (local prediction rigidity)
outputs = calculator.run_model(
    copper,
    outputs={"energy_uncertainty": ModelOutput(unit="eV", per_atom=True)},
)
local_uncertainty = outputs["energy_uncertainty"].block().values.squeeze(-1).cpu().numpy()
print("Local energy uncertainties (eV):", local_uncertainty)

Let's now try to see how the values change when we induce changes to the structure. First, let's rattle the atoms within the bulk configuration as shown below:

In [ ]:
print("Rattling the atoms!")
copper.rattle(1.0)

We can recompute the global and local uncertainties for this modified structure now:

In [ ]:
# Get global uncertainty after rattling
outputs = calculator.run_model(
    copper,
    outputs={"energy_uncertainty": ModelOutput(unit="eV", per_atom=False)},
)
global_uncertainty = outputs["energy_uncertainty"].block().values.item()
print("Global energy uncertainty after rattling (eV):", global_uncertainty)

In [ ]:
# Get local uncertainties after rattling
outputs = calculator.run_model(
    copper,
    outputs={"energy_uncertainty": ModelOutput(unit="eV", per_atom=True)},
)
local_uncertainty = outputs["energy_uncertainty"].block().values.squeeze(-1).cpu().numpy()
print("Local energy uncertainties after rattling (eV):", local_uncertainty)

We can see that, after rattling the structure quite aggressively, the uncertainties have gone up significantly. Nice!